In [1]:
from datetime import datetime, timedelta, date
from typing import Annotated, Union

from fastapi import Depends, FastAPI, HTTPException, status
from fastapi.security import OAuth2PasswordBearer, OAuth2PasswordRequestForm
from passlib.context import CryptContext
from pydantic import BaseModel, EmailStr
from enum import Enum
import psycopg2
import pandas as pd

In [23]:
DB_CON = {
        "host"      : 'rds-ticket-app-prod.czgosi3nm3l7.eu-central-1.rds.amazonaws.com',
        "database"  : 'public',
        "user"      : 'ticketappuser',
        "password"  : 'GA8yDbApaTthvqgH16V0jg==',
        'port'      : '5432'
    }

def format_value_for_sql(value):
    if value is None:
        return "NULL"
    elif isinstance(value, int):
        return str(value)
    elif isinstance(value, float):
        return str(value)
    elif isinstance(value, bool):
        return str(int(value))  # Convert bool to 0 or 1
    else:
        # If the data type is not recognized, raise an error or handle it accordingly
        return f"""'{str(value).replace("'", "''")}'"""

def read_data(sql):
    conn = psycopg2.connect(**DB_CON)
    cur = conn.cursor()
    cur.execute(sql)
    rows = cur.fetchall()
    cols = [str(col[0]) for col in cur.description ]
    conn.close()
    return pd.DataFrame(rows, columns=cols)

In [24]:
sql = '''select * from  public."Form"'''
d = read_data(sql)
d

,processId,pickDate,bikeSize,bikeId,contractStartDate,bikeRrp,contractAmount,contractSettlementPrice,contractSettlementDate,status,...,userFirstName,userFunction,userGender,userHouseNumber,userLastName,userMobilePhone,userStreet,userTelephone,userZipCode,bikeType
0,903550,2023-08-30,,NaN,2022-03-25,3283.95,3283.95,NaN,2023-08-18 00:00:00.000,InPickUp,...,Carsten,,Mr,1,Zoll,,Dieselstr.,05171/7009-140,31228,Pedelec
1,903551,2023-08-14,,NaN,2023-03-28,4447.86,4447.86,1708.05,2023-07-31 00:00:00.000,InboundCheck,...,Arif,,Mr,101A,Bilgen,,Kaiserstraße,01736343092,52146,Bicycle
2,903552,2023-08-10,,NaN,2022-05-25,5763.99,5763.99,2219.29,2023-07-28 00:00:00.000,InboundCheck,...,Bastian,,Mr,9a,Althof,,Himburg,0170/2729094,56459,Pedelec
3,903553,2023-08-14,,NaN,2022-05-09,4763.92,4763.92,NaN,2023-08-03 00:00:00.000,InboundCheck,...,Elvira,,Mrs,3 a,Schulz,,Bahnhofstr.,078348338919,77709,Pedelec
4,903554,2023-08-30,,NaN,2023-03-25,2399.00,2399.00,NaN,2023-08-18 00:00:00.000,InPickUp,...,Carsten,,Mr,1,Braun,,Dieselstraße,05171/7009-140,31228,Pedelec
5,903555,2023-07-11,,235540.0,2022-09-15,3129.85,3129.85,1201.94,2023-07-03 00:00:00.000,InboundCheck,...,Frau,,Mrs,57,Hoffmann,,Lehmkuhlenweg,051719063414,31224,Pedelec
6,903556,2023-07-11,,235541.0,2021-09-16,3299.00,3299.00,914.62,2023-07-03 00:00:00.000,InboundCheck,...,Frau,,Mrs,26,Martin,,Albert-Einstein-Str.,05031960610,31515,Pedelec
7,903557,2023-08-15,,NaN,2021-07-19,4012.95,4012.95,1162.37,2023-07-03 00:00:00.000,InboundCheck,...,Frau,,Mrs,6,Böttcher,,Am Eichelbach,05521894319,37412,Pedelec
8,903559,2023-07-25,,NaN,2022-10-06,3777.72,3777.72,1479.44,2023-07-04 00:00:00.000,InboundCheck,...,Robin,,Mr,9,Pietsch,,Lauske,035933-30690 oder 0171-2600437,2699,Pedelec
9,903560,2023-08-15,,NaN,2022-09-09,3538.90,3538.90,1343.27,2023-07-11 00:00:00.000,InboundCheck,...,Sandra,,Mrs,5 - 9,Maack,,Auestraße,+49 7836 51-41066,77761,Pedelec


In [32]:
import json
json.loads(d.to_json(orient = 'records'))

[{'processId': 903550,
  'pickDate': 1693353600000,
  'bikeSize': '',
  'bikeId': None,
  'contractStartDate': 1648166400000,
  'bikeRrp': 3283.95,
  'contractAmount': 3283.95,
  'contractSettlementPrice': None,
  'contractSettlementDate': 1692316800000,
  'status': 'InPickUp',
  'pickUpLogistics': 'Wiechert',
  'statusComment': 'Bitte so schnell wie möglich Kontakt zum Kunden aufnehmen bzgl. Abholung!',
  'submittingUser': 'modulat@rebike.com',
  'contractNumber': 'L656426/ A330',
  'pickUpFeedback': '',
  'bikeAccessories': 'none',
  'bikeAdditionalText': '',
  'bikeBatteryNumber': '',
  'bikeBrand': 'Haibike',
  'bikeColor': 'Red',
  'bikeFrameNumber': 'SJ20128286',
  'bikeKind': '',
  'bikeModel': 'Trekking 7',
  'bikeModelYear': None,
  'bikeOriginalPurchasePrice': 3283.95,
  'bikeOriginalPurchasePriceNet': None,
  'bikeUser': None,
  'contractAttachment': '',
  'contractCommunicationError': 'None',
  'contractConditionBasedReduction': None,
  'contractDeliveryInformation': '',
  

In [10]:
class Role(str, Enum):
    admin = "admin"
    user = "user"

class User(BaseModel):
    username: str
    email: EmailStr
    role: Role
    name: Union[str, None] = None
    disabled: Union[bool, None] = None

In [17]:
role = Role('admin')
user = User(username='med', email='med@rebike.com', role=role, name='medd', disabled=False)
user

User(username='med', email='med@rebike.com', role=<Role.admin: 'admin'>, name='medd', disabled=False)

AttributeError: 'User' object has no attribute 'value'